In [29]:
#将属性名按照标点，分割成token
import re
def snake_case_split(line):
    #Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
    #r表示非转义的原始字符串
    #由于正则表达式通常都包含反斜杠，所以你最好使用原始字符串来表示它们。模式元素(如 r'\t'，等价于 '\\t')匹配相应的特殊字符。
    #[...]用来表示一组字符,单独列出：[amk] 匹配 'a'，'m'或'k'
    line_split = re.split(r'[\s_]',line.strip())
    line_split = [line.strip() for line in line_split if len(line.strip())>0]
    return line_split

str = "First_Name"
print(snake_case_split(str))


#将属性按照大写字母，分割成token
import re
def camel_case_split(str):
    if "a"<=str[0]<="z" :
        strTemp=str[0].upper()+str[1:]
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', strTemp)
    else :
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

str = "FirstName"
print(camel_case_split(str))


#将属性名先按照snakeCase拆分，再按照camelCase拆分
def attributeToToken(str):
    tokenList1 = snake_case_split(str)
    tokenList2 = []
    for snakeToken in tokenList1:
        camelTokenList = camel_case_split(snakeToken)
        for token in camelTokenList:
            tokenList2.append(token[0].lower()+token[1:])
    return tokenList2

str = "FirstName_LastName"
list=attributeToToken(str)

['First', 'Name']


In [35]:
#加载模型
import fasttext
fasttext.FastText.eprint = lambda x: None
model = fasttext.load_model("/usr/dataLake/result/fil9.bin")


#将属性名的tokenList转换为词嵌入向量
def tokenToVector(tokenList):
    wordEmbedding=model.get_word_vector(tokenList[0])
    length= len(tokenList)
    for token in tokenList[1:]:
        wordEmbedding=wordEmbedding+model.get_word_vector(token)
    return wordEmbedding/length

In [38]:
tokenList=["first","name"]
w=tokenToVector(tokenList)
w

array([ 0.26734224,  0.15887718,  0.17169274, -0.4282757 , -0.18615206,
       -0.13033642, -0.07404496,  0.07804422, -0.35618225, -0.12453329,
       -0.08181277,  0.26787636, -0.07058604,  0.01830482,  0.11167918,
       -0.31116903,  0.12473547,  0.21723196,  0.22184291,  0.03101896,
        0.13705194, -0.05638621,  0.01378235,  0.03924557,  0.01057926,
       -0.30158573,  0.23949865,  0.21460846,  0.06502027,  0.33148494,
        0.08890073,  0.07159315,  0.22459815, -0.14856343, -0.04093441,
        0.02196516, -0.02158085,  0.07680135,  0.08894321,  0.3914056 ,
        0.38259572,  0.30266553,  0.15197313,  0.05165072,  0.04343948,
       -0.5956884 , -0.01191438,  0.29723346,  0.00325351,  0.02289635,
        0.02271349, -0.14330158,  0.11096019, -0.17221461,  0.26415366,
        0.03461661, -0.1272608 , -0.04122745,  0.08267218, -0.09647307,
        0.2005811 ,  0.22294575,  0.01480081, -0.11560582,  0.33165023,
        0.20925225,  0.03384192, -0.05086027, -0.1735389 ,  0.02

In [ ]:
class Attribute:
    def __init__(self, attributeName, originateFrom):
        self.attributeName = attributeName
        self.tokenList = attributeToToken(attributeName)
        #属性的词嵌入表示
        self.wordEmbedding = tokenToVector(self.tokenList)
        #属性来自哪张表
        self.originateFrom = originateFrom

In [ ]:
class Table:

    def __init__(self, ):

In [3]:
import pandas as pd

df = pd.read_csv('/usr/dataLake/datasets/BenchmarkCsvfiles/CTA_Ridership_Avg_Weekday_Bus_Stop_Boardings_in_October_2012____c5_0____1.csv')

#list函数返回一个表的所有属性，返回一个list
print(list(df))

['alightings', 'boardings', 'cross_street', 'location', 'routes']


In [ ]:
import os
import pandas as pd
#for循环中的info代表文件名
attributeList = []
for info in os.listdir('/usr/DataLake/datasets/BenchmarkCsvfiles'): 
    #去掉.csv后缀
    tableName = info[0:-4]
    domain = os.path.abspath('/usr/DataLake/datasets/BenchmarkCsvfiles') #获取文件夹的路径 
    info = os.path.join(domain,info) #将路径与文件名结合起来就是每个文件的完整路径
    df = pd.read_csv(info)   
    for attributeName in list(df):
        x=Attribute(attributeName, tableName)
        #通过append函数将自定义类的对象放入list中
        attributeList.append(x)
        #print(attributeList[i])可证明是Attribute类的对象


In [ ]:
a=0
while a<20:
    print(attributeList[a].attributeName)
    print(attributeList[a].originateFrom)
    a=a+1

In [ ]:
#nt小，ITF大，权重大。
#token越稀有，权重越大。
#N是数据湖中有多少列。